In [153]:
import numpy as np
from dataclasses import dataclass
from functools import cache
from itertools import zip_longest
import sys

%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [154]:
@dataclass
class Piece():
  rows: np.ndarray[np.int8, np.int8]
  columns: np.ndarray[np.int8, np.int8]

  def __repr__(self):
    buffer = []
    for row, column in zip_longest(self.rows, self.columns):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else ' ' for elem in row))
        if column is not None:
          buffer.append(" ".join('|' if elem else ' ' for elem in column))
    return "\n".join(buffer)
  
  def __hash__(self):
     return hash((str(self.rows), str(self.columns)))

  def __eq__(self, other):
      if isinstance(other, Piece):
          return np.array_equal(self.rows, other.rows) and np.array_equal(self.columns, other.columns)
      return NotImplemented
    
  def rotated(self):
    rows = np.rot90(self.columns, 1)
    columns = np.rot90(self.rows, 1)

    while ~np.any(rows[0]) and ~np.any(columns[0]):
      # top row is empty, so we can shift digit up
      rows = np.roll(rows, -1, 0)
      columns = np.roll(columns, -1, 0)

    return Piece(rows, columns)
  
  def trimmed(self):
    rows = self.rows
    columns = self.columns

    while rows.size and ~np.any(rows[-1]) and columns.size and ~np.any(columns[-1]):
       rows = rows[:-1]
       columns = columns[:-1]

    while rows.size and ~np.any(rows[:,-1]) and columns.size and ~np.any(columns[:,-1]):
       rows = rows[:,:-1]
       columns = columns[:,:-1]
    
    return Piece(rows, columns)
  
  @cache
  def rotations(self):
    a = self.trimmed()
    b = a.rotated().trimmed()
    c = b.rotated().trimmed()
    d = c.rotated().trimmed()
    return set((a, b, c, d))

pieces = [
  Piece(rows=np.asarray(((True, False), (True, False), (False, False)), dtype=np.int8), columns=np.asarray(((True, True, False), (False, False, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((False, False), (False, False), (False, False)), dtype=np.int8), columns=np.asarray(((True, False, False), (True, False, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((False, True, False), (True, False, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((False, True, False), (False, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((False, False), (True, False), (False, False)), dtype=np.int8), columns=np.asarray(((True, True, False), (False, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((True, False, False), (False, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((True, False, False), (True, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (False, False), (False, False)), dtype=np.int8), columns=np.asarray(((False, True, False), (False, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((True, True, False), (True, True, False)), dtype=np.int8)),
  Piece(rows=np.asarray(((True, False), (True, False), (True, False)), dtype=np.int8), columns=np.asarray(((True, True, False), (False, True, False)), dtype=np.int8)),
]
pieces = [piece.trimmed() for piece in pieces]

print(sys.getsizeof(pieces[0].rows))

128


In [155]:
pieces[0]

 -
| |
 -

In [156]:
pieces[1].trimmed()

 
|
 
|
 

In [157]:
pieces[9].rotations()

{   -
 | | |
  - -,
  -
 |  
  -
 | |
  -,
  -
 | |
  -
   |
  -,
  - -
 | | |
  -  }

In [158]:
pieces[0].rotations()

{ -
 | |
  -}

In [159]:
class Puzzle():
  rows: np.ndarray[np.int8, np.int8]
  cols: np.ndarray[np.int8, np.int8]

  def __init__(
      self,
      rows=np.zeros((5,5), dtype=np.int8),
      cols=np.zeros((4,6), dtype=np.int8),
    ):
    self.rows = rows
    self.cols = cols

  def __repr__(self):
    buffer = []
    for row, column in zip_longest(self.rows, self.cols):
        if row is not None:
          buffer.append(" " + " ".join('-' if elem else '.' for elem in row))
        if column is not None:
          buffer.append(" ".join('|' if elem else '.' for elem in column))
    return "\n".join(buffer)

  def __hash__(self):
    return hash((str(self.rows), str(self.cols)))

  def __eq__(self, other):
    if isinstance(other, Puzzle):
        return np.array_equal(self.rows, other.rows) and np.array_equal(self.cols, other.cols)
    return NotImplemented

  def with_piece(self, piece: Piece, toprow: int, topcol: int) -> "Puzzle":
    # This assumes the piece is trimmed
    if (toprow + piece.rows.shape[0] > self.rows.shape[0]):
      return None
    if (topcol + piece.rows.shape[1] > self.rows.shape[1]):
      return None
    if (toprow + piece.columns.shape[0] > self.cols.shape[0]):
      return None
    if (topcol + piece.columns.shape[1] > self.cols.shape[1]):
      return None
    
    puzzle_rows = self.rows[toprow:toprow+piece.rows.shape[0], topcol:topcol+piece.rows.shape[1]]
    piece_rows_nonzero = np.nonzero(piece.rows)
    if np.any(puzzle_rows[piece_rows_nonzero]):
      return None

    puzzle_cols = self.cols[toprow:toprow+piece.columns.shape[0], topcol:topcol+piece.columns.shape[1]]
    piece_cols_nonzero = np.nonzero(piece.columns)
    if np.any(puzzle_cols[piece_cols_nonzero]):
      return None
    
    new_rows = self.rows.copy()
    new_cols = self.cols.copy()
    puzzle_rows = new_rows[toprow:toprow+piece.rows.shape[0], topcol:topcol+piece.rows.shape[1]]
    puzzle_cols = new_cols[toprow:toprow+piece.columns.shape[0], topcol:topcol+piece.columns.shape[1]]
    puzzle_rows[piece_rows_nonzero] = piece.rows[piece_rows_nonzero]
    puzzle_cols[piece_cols_nonzero] = piece.columns[piece_cols_nonzero]

    return Puzzle(new_rows, new_cols)
  
  def spaces_left(self):
    return np.size(self.cols) - np.count_nonzero(self.cols) + np.size(self.rows) - np.count_nonzero(self.rows)

In [160]:
%timeit Puzzle().with_piece(pieces[7], 2, 3)

38.8 µs ± 5.41 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [161]:
%lprun -f Puzzle.with_piece Puzzle().with_piece(pieces[7], 2, 3)

Timer unit: 1e-07 s

Total time: 0.0002037 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_740\2496104955.py
Function: with_piece at line 30

Line #      Hits         Time  Per Hit   % Time  Line Contents
    30                                             def with_piece(self, piece: Piece, toprow: int, topcol: int) -> "Puzzle":
    31                                               # This assumes the piece is trimmed
    32         1         57.0     57.0      2.8      if (toprow + piece.rows.shape[0] > self.rows.shape[0]):
    33                                                 return None
    34         1         17.0     17.0      0.8      if (topcol + piece.rows.shape[1] > self.rows.shape[1]):
    35                                                 return None
    36         1         18.0     18.0      0.9      if (toprow + piece.columns.shape[0] > self.cols.shape[0]):
    37                                                 return None
    38         1         17.0     17.0     

In [162]:
Puzzle()

 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [163]:
Puzzle().spaces_left()

49

In [164]:
Puzzle().with_piece(pieces[0], 1, 4)

 . . . . .
. . . . . .
 . . . . -
. . . . | |
 . . . . -
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [165]:
Puzzle().with_piece(pieces[7].rotated(), 0, 0)

 - - . . .
| . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [166]:
Puzzle().with_piece(pieces[3], 0, 0).with_piece(pieces[4].rotated(), 0, 1)

 - - - . .
. | | . . .
 - - . . .
. | . . . .
 - . . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [167]:
Puzzle().with_piece(pieces[9].rotated().rotated().rotated(), 1, 0)

 . . . . .
. . . . . .
 . - . . .
| | | . . .
 - - . . .
. . . . . .
 . . . . .
. . . . . .
 . . . . .

In [168]:
from copy import copy

bad_puzzles = set()

# pick a puzzle piece
# choose rotation
# find top-left free corner
# put piece if it fits
# check for solved
def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
  if puzzle in bad_puzzles:
    return None
  if puzzle.spaces_left() == 2:
    return puzzle, used_pieces
  
  candidates = []
  for i in range(len(pieces)):
    if i in used_pieces:
      continue

    for rotation in pieces[i].rotations():
      for row in range(0, len(puzzle.rows)):
        for col in range(0, len(puzzle.cols)):
          candidate = puzzle.with_piece(rotation, row, col)
    
          if candidate is not None:
            if candidate in bad_puzzles:
              continue

            candidate_used_pieces = copy(used_pieces)
            candidate_used_pieces[i] = (rotation, row, col)
            candidates.append((candidate, candidate_used_pieces))
    
  for candidate, candidate_used_pieces in candidates:
    result = attempt(candidate, candidate_used_pieces)
    if result is not None:
      return result

  bad_puzzles.add(puzzle)
  return None

In [169]:
rot9 = pieces[9].rotated().rotated().rotated()
puzzle = Puzzle().with_piece(rot9, 1, 0)
%lprun -f attempt attempt(puzzle, {9: (rot9, 1, 0)})

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-07 s

Total time: 184.996 s
File: C:\Users\Vidminas\AppData\Local\Temp\ipykernel_740\3240605786.py
Function: attempt at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def attempt(puzzle: Puzzle, used_pieces: dict[int, tuple]):
    11     14470  154811842.0  10698.8      8.4    if puzzle in bad_puzzles:
    12                                               return None
    13     14470    3567816.0    246.6      0.2    if puzzle.spaces_left() == 2:
    14                                               return puzzle, used_pieces
    15                                             
    16     14470      69612.0      4.8      0.0    candidates = []
    17    159167    1123881.0      7.1      0.1    for i in range(len(pieces)):
    18    144698    1022388.0      7.1      0.1      if i in used_pieces:
    19     94842     349899.0      3.7      0.0        continue
    20                                      

In [170]:
len(bad_puzzles)

14465

In [171]:
next(iter(bad_puzzles))

 - . . . .
| | . | . .
 - - . . .
| | | | . .
 - - . . .
. | | | . .
 . - - . .
. . . | . .
 . - - . .